In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import random
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold
from lightgbm import LGBMClassifier

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn import preprocessing
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')

In [ ]:
train.head()

------NO NULL VALUES------


In [ ]:
categorical_cols = ['cat'+str(i) for i in range(19)]
continous_cols = ['cont'+str(i) for i in range(11)]

In [ ]:
cols=categorical_cols+continous_cols
train_objs_num = len(train)
dataset = pd.concat(objs=[train[cols], test[cols]], axis=0)
dataset_preprocessed = pd.get_dummies(dataset,columns=categorical_cols)
train_preprocessed = dataset_preprocessed[:train_objs_num]
test_preprocessed = dataset_preprocessed[train_objs_num:]

In [ ]:
train_preprocessed.head()

In [ ]:
from tensorflow.keras.layers import Input,Dense,Dropout
from tensorflow.keras import Model
from  tensorflow.keras.regularizers import l2
import tensorflow as tf

In [ ]:
len(continous_cols)

In [ ]:
def get_DAE():
    # denoising autoencoder
    inputs = Input((11,))
    x = Dense(1500, activation='relu')(inputs) # 1500 original
    x = Dense(1500, activation='relu', name="feature")(x) # 1500 original
    x = Dense(1500, activation='relu')(x) # 1500 original
    outputs = Dense(11, activation='relu')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')

    return model

In [ ]:
alldata = pd.concat([train[continous_cols],test[continous_cols]],axis=0)
print(alldata.shape)
autoencoder = get_DAE()
autoencoder.fit(alldata[continous_cols], alldata[continous_cols],
                    epochs=20,
                    batch_size=256,
                    shuffle=True
                    )

In [ ]:
test_denoised = test_preprocessed.copy()
test_denoised[continous_cols] = autoencoder.predict(test_denoised[continous_cols])
train_denoised = train_preprocessed.copy()
train_denoised[continous_cols] = autoencoder.predict(train_denoised[continous_cols])

In [ ]:
train_denoised['target'] = train.target

In [ ]:
X_train = train_denoised.drop(['target'], axis=1)
y_train = train_denoised.target
X_test = test_denoised

In [ ]:
params={'metric': 'auc', 'reg_alpha': 6.010538011450937, 'reg_lambda': 0.031702113663443346, 'colsample_bytree': 0.27,
   'subsample': 0.6, 'learning_rate': 0.05, 'max_depth': 100, 'num_leaves': 100, 'min_child_samples': 216,
   'cat_smooth': 87, 'random_state': 48,'n_estimators': 20000}

In [ ]:
preds = np.zeros(test.shape[0])        
kf = StratifiedKFold(n_splits=10,random_state=48,shuffle=True)                  
auc=[]   # list contains AUC for each fold  
n=0   
for trn_idx, test_idx in kf.split(X_train,train['target']):
    X_tr,X_val=X_train.iloc[trn_idx],X_train.iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    model = LGBMClassifier(**params) 
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False) 
    preds+=model.predict_proba(X_test)[:, 1]/kf.n_splits 
    auc.append(roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])) 
    print(n+1,auc[n])                                                                                       
    n+=1

In [ ]:
np.mean(auc)

In [ ]:
sub['target']=preds
sub.to_csv('submission.csv', index=False)

In [ ]:
sub